# Using the Vertex AI PaLM API to Generate and Ideate

# Core code
Let's define some variables that will be used throughout this notebook.

These are the GCP Project ID `project_id`, the Model name `model_name` which is any name you prefer, and finally the Dataset name `dataset_name`.
The dataset needs to exist in the same Project as `project_id` and you'll need appropriate access to create and delete.

## Setup

In [4]:
from typing import Union
import sys

import os
import io
import json
import base64
import requests
import concurrent.futures
import time

import numpy as np
import pandas as pd

import vertexai
import vertexai.language_models as language_models
import vertexai.preview.language_models as language_models_preview
from google.cloud import aiplatform
from google.cloud import documentai
from google.cloud.documentai_v1 import Document
from google.cloud import storage
from google.cloud import bigquery

from IPython.display import display, Markdown, Latex


In [5]:
PROJECT_ID = 'mg-ce-demos'
REGION = 'us-central1'


In [6]:
# vertex ai clients
vertexai.init(project = PROJECT_ID, location = REGION)
aiplatform.init(project = PROJECT_ID, location = REGION)


## Ideate with Vertex AI PaLM API

In [7]:
textgen_model = language_models_preview.TextGenerationModel.from_pretrained('text-bison-32k')

### Now, we send our prompt to Google GenAI API for some LLM magic

In [10]:
# Provide some context
pre_text = "Pretend you're a marketing specialist, "

# Ask the LLM
text = "Generate a social media post for a new manufacturing robot product"

prompt = pre_text+text

In [11]:
display(Markdown('## Prompt:'))
print(prompt)
display(Markdown('## Response:'))

# Send prompt to LLM
display(Markdown(str(textgen_model.predict(
   (prompt),
    max_output_tokens=1024,
    temperature=0.4,
    top_p=0.8,
    top_k=40,
))))

## Prompt:

Pretend you're a marketing specialist, Generate a social media post for a new manufacturing robot product


## Response:

 Introducing the new manufacturing robot product, designed to revolutionize the manufacturing industry. This cutting-edge robot is equipped with the latest technology, including artificial intelligence and machine learning, to optimize production processes and increase efficiency. With its sleek design and user-friendly interface, this robot is the perfect addition to any manufacturing facility. Get ready to take your production to the next level with this revolutionary new product.
#manufacturing #robot #technology #innovation #efficiency #productivity

In [12]:
def marketing_bot(text):
    pre_text = "Pretend you're a marketing specialist, "
    prompt = pre_text + text
    result = textgen_model.predict(
        prompt,
        max_output_tokens=1024,
        temperature=0.4,
        top_p=0.8,
        top_k=40,
    )
    
    return prompt, result

## Now we can stick it behind a UI

In [13]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown(
    """
    ## Marketing Bot

    ### Enter a task...

    """)
    with gr.Row():
      with gr.Column():
        input_text = gr.Textbox(label="Task", placeholder="Pretend you're a marketing specialist, ")
        
        
    with gr.Row():
      generate = gr.Button("Generate Response")

    with gr.Row():
      label2 = gr.Textbox(label="Prompt")
    with gr.Row():
      label3 = gr.Textbox(label="Response generated by LLM")

    generate.click(marketing_bot,input_text, [label2, label3])
demo.launch(share=False, debug=False)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
